to access the openAI API, you need an API key which you can get it from  https://platform.openai.com/api-keys
once you have a key then you can either save it as an environment variable or hard code in as needed from simplicity

In [16]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv(override=True)
#os.environ["OPENAI_API_KEY"] = "your key"



True

In [17]:
import os

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4-0125-preview")
output_parser = StrOutputParser()


Use document loaders to load data from a source as Document

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [23]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os

raw_documents = TextLoader('../article/funnyIO_development.md').load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

vectorstore = FAISS.from_documents(documents, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]))

retriever = vectorstore.as_retriever()

In [ ]:
from pprint import pprint

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt 
    | model 
    | output_parser
)

pprint(chain.invoke("tell me about the funnyIO"))

you can also load directory using directory loader.
and split a long document into smaller chunks that can fit into your model's context window.

In [ ]:
%pip install markdown

In [31]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PythonLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


#raw_documents = DirectoryLoader('../article', glob="*.py", loader_cls=PythonLoader).load()
#raw_documents = DirectoryLoader('../article', glob="*.txt", loader_cls=TextLoader).load()
raw_documents = DirectoryLoader('../article', glob="*.md", loader_cls=UnstructuredMarkdownLoader).load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

vectorstore = FAISS.from_documents(documents, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]))

retriever = vectorstore.as_retriever()


invoke question

In [32]:
from pprint import pprint

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt 
    | model 
    | output_parser
)

pprint(chain.invoke("tell me about the emerging funnnnyIO"))

('Emerging FunnyIO technologies include advancements that are revolutionizing '
 'data transfer and processing capabilities. Among these technologies are '
 'USB4, Thunderbolt 4, PCIe 5.0, and FunnyIO 6.0. These advancements are '
 'crucial for meeting the increasing demands of modern computing environments, '
 'allowing for enhanced productivity, streamlined workflows, and unlocking new '
 'possibilities in data exchange. USB4, for instance, is highlighted as a '
 'significant improvement in the Universal Serial Bus standard, offering data '
 'transfer speeds of up to 40 Gbps over a single cable. This is particularly '
 'beneficial for connecting external storage devices, displays, and other '
 'peripherals requiring high bandwidth, thus facilitating faster backup and '
 'file transfer speeds and enhancing user productivity with large data '
 'volumes. These emerging high-speed I/O technologies are essential in keeping '
 'pace with the growing need for efficient and reliable data tra